In [1]:
import numpy as np
import pandas as pd
import os, time, random

In [2]:
#Read the dataset prepared after feature engineering
data_path = 'data.csv'
data = pd.read_csv(data_path)

#Understanding the dataframe and datatypes for each column
print(data.info())

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (11,12,13,14,16,17,18,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4883327 entries, 0 to 4883326
Data columns (total 47 columns):
 #   Column              Dtype  
---  ------              -----  
 0   id                  object 
 1   item_id             object 
 2   dept_id             object 
 3   cat_id              object 
 4   store_id            object 
 5   state_id            object 
 6   d                   object 
 7   sales               float64
 8   release             int64  
 9   wm_yr_wk            int64  
 10  date                object 
 11  event_name_1        object 
 12  event_type_1        object 
 13  event_name_2        object 
 14  event_type_2        object 
 15  snap_TX             int64  
 16  event_name_1_lag_1  object 
 17  event_type_1_lag_1  object 
 18  event_name_1_lag_2  object 
 19  event_type_1_lag_2  object 
 20  event_name_1_lag_3  object 
 21  event_type_1_lag_3  object 
 22  snap_TX_lag_1       float64
 23  snap_TX_lag_2       float64
 24  snap_TX_lag_3       floa

In [3]:
#remove columns which might lead to ordinal behavior

remove_features = ['id','item_id','dept_id','cat_id','state_id','store_id', 'release', 'event_name_1','event_type_1','event_name_2','event_type_2','event_name_1_lag_1', 'event_type_1_lag_1',
                   'event_name_1_lag_2', 'event_type_1_lag_2', 'event_name_1_lag_3', 'event_type_1_lag_3',
                   'date','wm_yr_wk','d','sales','temp_d','day', 'week', 'month', 'year', 'dayofweek', 'weekend']

In [4]:
#Convert the object columns to datatype 'category'
category_columns=['id','item_id','dept_id','cat_id','store_id','state_id','event_name_1','event_type_1','event_name_2','event_type_2','event_name_1_lag_1', 'event_type_1_lag_1',
                   'event_name_1_lag_2', 'event_type_1_lag_2', 'event_name_1_lag_3', 'event_type_1_lag_3']

#Convert each category in the list one 
for col in category_columns:
    data[col] = data[col].astype('category')

#Create a list of Store Ids for which data is considered 
stores_ids = data['store_id']
stores_ids = list(stores_ids.unique())


In [5]:
#Check if it is indeed 'TX_1', since we choose this specific store for modeling purposes
#due to processing power limitations and to avoid OOM(Out of Memory) Error 
stores_ids

['TX_1']

In [6]:
#copy the dataframe into new df
df = data.copy()
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,...,lag_34,lag_35,rolling_mean_7,rolling_std_7,rolling_mean_14,rolling_std_14,rolling_mean_30,rolling_std_30,rolling_mean_60,rolling_std_60
0,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX,d_1,1.0,11101,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_008_TX_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,TX_1,TX,d_1,4.0,11101,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_009_TX_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,TX_1,TX,d_1,3.0,11101,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_010_TX_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,TX_1,TX,d_1,0.0,11101,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_012_TX_1_evaluation,HOBBIES_1_012,HOBBIES_1,HOBBIES,TX_1,TX,d_1,0.0,11101,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883322,FOODS_3_823_TX_1_evaluation,FOODS_3_823,FOODS_3,FOODS,TX_1,TX,d_1941,NaN,11101,11617,...,0.0,0.0,0.4285,0.5347,0.5000,0.6504,0.56700,0.6260,0.3167,0.5366
4883323,FOODS_3_824_TX_1_evaluation,FOODS_3_824,FOODS_3,FOODS,TX_1,TX,d_1941,NaN,11101,11617,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.03333,0.1826,0.0500,0.2197
4883324,FOODS_3_825_TX_1_evaluation,FOODS_3_825,FOODS_3,FOODS,TX_1,TX,d_1941,NaN,11101,11617,...,1.0,2.0,0.5713,0.5347,0.9287,0.8286,0.53300,0.7760,0.4666,0.7240
4883325,FOODS_3_826_TX_1_evaluation,FOODS_3_826,FOODS_3,FOODS,TX_1,TX,d_1941,NaN,11312,11617,...,0.0,0.0,0.4285,0.5347,0.5713,0.6460,0.83350,0.9497,0.7666,0.9272


In [7]:
#Selected categorical columns are used for OneHotEnconding or to create DummyVariables/Columns
#removes ordinal behavior  
df = pd.get_dummies(data=df, columns=['dept_id','event_name_1','event_name_2','day',
 'week',
 'month',
 'year',
 'dayofweek',
 'weekend'])

#create a temporary date column with integer values which denotes day number 
#this is later used for subsetting the data into test/train
df['temp_d'] = pd.to_numeric(data['d'].str[2:])

#Once selected categorical columns are dummy encoded, 
#create list of categorical columns to remove from df 
features = [col for col in list(df) if col not in remove_features]

In [8]:
#Checking dummy encoded columns
df

,id,item_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,date,...,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,weekend_0,weekend_1,temp_d
0,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES,TX_1,TX,d_1,1.0,11101,11101,2011-01-29,...,0,0,0,0,0,1,0,0,1,1
1,HOBBIES_1_008_TX_1_evaluation,HOBBIES_1_008,HOBBIES,TX_1,TX,d_1,4.0,11101,11101,2011-01-29,...,0,0,0,0,0,1,0,0,1,1
2,HOBBIES_1_009_TX_1_evaluation,HOBBIES_1_009,HOBBIES,TX_1,TX,d_1,3.0,11101,11101,2011-01-29,...,0,0,0,0,0,1,0,0,1,1
3,HOBBIES_1_010_TX_1_evaluation,HOBBIES_1_010,HOBBIES,TX_1,TX,d_1,0.0,11101,11101,2011-01-29,...,0,0,0,0,0,1,0,0,1,1
4,HOBBIES_1_012_TX_1_evaluation,HOBBIES_1_012,HOBBIES,TX_1,TX,d_1,0.0,11101,11101,2011-01-29,...,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883322,FOODS_3_823_TX_1_evaluation,FOODS_3_823,FOODS,TX_1,TX,d_1941,NaN,11101,11617,2016-05-22,...,0,0,0,0,0,0,1,0,1,1941
4883323,FOODS_3_824_TX_1_evaluation,FOODS_3_824,FOODS,TX_1,TX,d_1941,NaN,11101,11617,2016-05-22,...,0,0,0,0,0,0,1,0,1,1941
4883324,FOODS_3_825_TX_1_evaluation,FOODS_3_825,FOODS,TX_1,TX,d_1941,NaN,11101,11617,2016-05-22,...,0,0,0,0,0,0,1,0,1,1941
4883325,FOODS_3_826_TX_1_evaluation,FOODS_3_826,FOODS,TX_1,TX,d_1941,NaN,11312,11617,2016-05-22,...,0,0,0,0,0,0,1,0,1,1941


In [9]:
#List of features that we are finally considering for Modeling
features

['snap_TX',
 'snap_TX_lag_1',
 'snap_TX_lag_2',
 'snap_TX_lag_3',
 'lag_28',
 'lag_29',
 'lag_30',
 'lag_31',
 'lag_32',
 'lag_33',
 'lag_34',
 'lag_35',
 'rolling_mean_7',
 'rolling_std_7',
 'rolling_mean_14',
 'rolling_std_14',
 'rolling_mean_30',
 'rolling_std_30',
 'rolling_mean_60',
 'rolling_std_60',
 'dept_id_FOODS_1',
 'dept_id_FOODS_2',
 'dept_id_FOODS_3',
 'dept_id_HOBBIES_1',
 'dept_id_HOBBIES_2',
 'dept_id_HOUSEHOLD_1',
 'dept_id_HOUSEHOLD_2',
 'event_name_1_Chanukah End',
 'event_name_1_Christmas',
 'event_name_1_Cinco De Mayo',
 'event_name_1_ColumbusDay',
 'event_name_1_Easter',
 'event_name_1_Eid al-Fitr',
 'event_name_1_EidAlAdha',
 "event_name_1_Father's day",
 'event_name_1_Halloween',
 'event_name_1_IndependenceDay',
 'event_name_1_LaborDay',
 'event_name_1_LentStart',
 'event_name_1_LentWeek2',
 'event_name_1_MartinLutherKingDay',
 'event_name_1_MemorialDay',
 "event_name_1_Mother's day",
 'event_name_1_NBAFinalsEnd',
 'event_name_1_NBAFinalsStart',
 'event_name_1_

In [10]:
#Creating variables for limiting the data by dates
START_TRAIN = 1000                # Start day for training data
END_TRAIN   = 1885               # End day of our train set, 28 days after this are left for testing(Start day for Testing Data)  
LimitData   = 1913              # End day for Testing Data
    
#Subset the data for 1000 to 1913 days
df = df[(df['temp_d']>=START_TRAIN) & (df['temp_d']<=LimitData)].reset_index(drop=True)

#df = df[(df['temp_d']>=START_TRAIN)].reset_index(drop=True)


In [11]:
#Create train and test datasets   
train_mask = df['temp_d']<=END_TRAIN
#valid_mask = train_mask&(df['temp_d']>(END_TRAIN-P_HORIZON))
preds_mask = df['temp_d']>(END_TRAIN)


train = df[train_mask.values]
test = df[preds_mask.values]

#Split both train and test datasets for independant and depandant variables 
x_train = train[features]
y_train = train[['sales']]


x_test = test[features]
y_test = test[['sales']]



In [12]:
#Fill the NAs with 0, if present

x_test1 = x_test
y_test1 = y_test

x_train1 = x_train
y_train1 = y_train

In [13]:
y_test1

,sales
2597874,1.0
2597875,0.0
2597876,0.0
2597877,0.0
2597878,0.0
...,...
2683241,0.0
2683242,0.0
2683243,0.0
2683244,0.0


In [ ]:
##Import the necessary libraries for - XG Boost

import xgboost as xgb

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore")



x_model = xgb.XGBRegressor(
 learning_rate =0.09,
 n_estimators=1200,
 max_depth=4,
 min_child_weight=3,
 gamma=0,
 subsample=0.8,
 reg_alpha=200, reg_lambda=200,
 colsample_bytree=0.8,nthread=4)

#Fit the model based on training data 
x_model.fit(x_train1,y_train1)


In [ ]:
# Import the Mean Squared Error to eventually calculate the RMSE

from sklearn.metrics import mean_squared_error as MSE
pred = x_model.predict(x_test1)
rmse = np.sqrt(MSE(y_test1, pred))

In [ ]:
rmse